<h1 style="color:#65AE11;">Memory Copies in Non-Default Streams</h1>

In this section you will perform host-to-device and device-to-host memory transfers in non-default streams.

<h2 style="color:#65AE11;">Objectives</h2>

By the time you complete this section you will:

* Know how to create pinned memory, which can be asynchronously transfered in non-default streams
* Be able to perform host-to-device memory transfers in non-default streams
* Be able to perform device-to-host memory transfers in non-default streams

<h2 style="color:#65AE11;">Allocating Pinned Memory</h2>

In order to asynchronously copy data, CUDA needs to make assumptions about its location. Typical host memory uses [paging](https://en.wikipedia.org/wiki/Paging) so that in addition to RAM, data can be stored on some backup storage device like a physical disk.

Pinning, or page-locking memory bypasses host OS paging, storing allocated memory in RAM. Page-locking, or pinning memory is required to transfer memory asynchronously in a non-default stream.

Because it prevents storage of data on some backup storage, pinned memory is a limited resource, and care must be taken not to over use it.

Pinned host memory is allocated with `cudaMallocHost`:

```c
const uint64_t num_entries = 1UL << 26;
uint64_t *data_cpu;
cudaMallocHost(&data_cpu, sizeof(uint64_t)*num_entries);
```

```c
const uint64_t num_entries = 1UL << 26;
uint64_t *data_cpu;
cudaMallocHost(&data_cpu, sizeof(uint64_t)*num_entries);
```

<h2 style="color:#65AE11;">Host-to-Device Memory Transfers in a Non-Default Stream</h2>

Pinned host memory can be transfered to GPU memory in a non-default stream using `cudaMemcpyAsync` which is similar to `cudaMemcpy` but expects a 5th stream identifier argument:

```c
cudaStream_t stream;
cudaStreamCreate(&stream);

const uint64_t num_entries = 1UL << 26;

uint64_t *data_cpu, *data_gpu;

cudaMallocHost(&data_cpu, sizeof(uint64_t)*num_entries);
cudaMalloc(&data_cpu, sizeof(uint64_t)*num_entries);

cudaMemcpyAsync(data_gpu, 
                data_cpu, 
                sizeof(uint64_t)*num_entries, 
                cudaMemcpyHostToDevice, 
                stream);
```

<h2 style="color:#65AE11;">Device-to-Host Memory Transfers in a Non-Default Stream</h2>

GPU memory can be also be transfered to pinned host memory in a non-default stream using `cudaMemcpyAsync`:

```c
// Assume data is already present on the GPU, and that `data_cpu` is pinned.

cudaMemcpyAsync(data_cpu, 
                data_gpu, 
                sizeof(uint64_t)*num_entries, 
                cudaMemcpyDeviceToHost, 
                stream);
```

As is the case with all modern GPUs, GPU devices with 2 or more copy engines can perform host-to-device and device-to-host memory transfers in different non-default streams at the same time. You will do this yourself later in the course.

<h2 style="color:#65AE11;">Stream Synchronization</h2>

Using `cudaStreamSynchronize` will cause host code to block until a given stream has completed its operations. Stream synchronization should be used when guarantees are needed about the completion of a stream's work, for example, when host code needs to wait for asynchronous memory transfers in a non-default stream to complete:

```c
// Assume data is already present on the GPU, and that `data_cpu` is pinned.

cudaMemcpyAsync(data_cpu, 
                data_gpu, 
                sizeof(uint64_t)*num_entries, 
                cudaMemcpyDeviceToHost, 
                stream);

// Block until work (in this case memory transfer to host) in `stream` is complete.
cudaStreamSynchronize(stream);

// `data_cpu` transfer to host via `stream` is now guaranteed to be complete.
checkResultCpu(data_cpu);
```

<h2 style="color:#65AE11;">Exercise: Perform Memory Transfers in Non-Default Stream</h2>

Open and refactor [*07_Memcpy_in_Streams/baseline_cipher/baseline.cu*](baseline_cipher/baseline.cu) to perform both host-to-device and device-to-host memory transfers in a non-default stream.

Generate a report file for the refactored application by using a JupyterLab terminal and running `make profile` from within the *07_Memcpy_in_Streams/baseline_cipher* directory. (See the [*Makefile*](baseline_cipher/Makefile) there for details).

Open the report file in Nsight Systems. If you've closed the Nsight Systems tab, you can reopen it by following the instructions in [*Nsight Systems Setup*](../04_Nsight_Systems_Setup/Nsight_Systems_Setup.ipynb). As a reminder the password is `nvidia`.

If you were successful, you should notice in the Nsight Systems visual timeline that memory transfers are now occuring in non-default streams, as is shown in the screenshot below.

If you get stuck, please refer to [07_Memcpy_in_Streams/baseline_cipher/baseline_solution.cu](../07_Memcpy_in_Streams/baseline_cipher/baseline_solution.cu).

In [3]:
%%bash
# Affiche le répertoire courant pour vérifier le point de départ
pwd

cd kernel_baseline_cipher

module load cuda/12.6

make profile

./baseline

/gpfs/home/scortinhal/CHPS0904/task/07_Memcpy_in_Streams
nsys profile --stats=true --force-overwrite=true -o baseline-report ./baseline
TIMING: 24.0244 ms (allocate memory)
TIMING: 15736.8 ms (encrypt data on CPU)
TIMING: 1.92035 ms (copy data from CPU to GPU)
TIMING: 33.8242 ms (decrypt data on GPU)
TIMING: 3.18275 ms (copy data from GPU to CPU)
TIMING: 39.0159 ms (total time on GPU)
STATUS: test passed
TIMING: 4.78464 ms (checking result on CPU)
TIMING: 11.6836 ms (free memory)
TIMING: 15816.6 ms (Temps total)
Generating '/tmp/nsys-report-2710.qdstrm'
[1/8] [========================100%] baseline-report.nsys-rep
[2/8] [========================100%] baseline-report.sqlite


SKIPPED: /gpfs/home/scortinhal/CHPS0904/task/07_Memcpy_in_Streams/kernel_baseline_cipher/baseline-report.sqlite does not contain NV Tools Extension (NVTX) data.


[3/8] Executing 'nvtx_sum' stats report
[4/8] Executing 'osrt_sum' stats report

 Time (%)  Total Time (ns)  Num Calls   Avg (ns)     Med (ns)    Min (ns)  Max (ns)   StdDev (ns)           Name         
 --------  ---------------  ---------  -----------  -----------  --------  ---------  -----------  ----------------------
     52.0      16081260032        171   94042456.3  100113984.0      1568  154654784   24151459.0  poll                  
     47.0      14524869280         63  230553480.6    4045184.0     32096  982468064  394648271.7  futex                 
      1.0        301383936        851     354152.7      13568.0      1088   80187232    3073977.6  ioctl                 
      0.0          6017472         38     158354.5       9184.0      2848    5612800     908761.4  mmap                  
      0.0          5737280         17     337487.1     319296.0    305568     475840      49178.4  pthread_create        
      0.0           716160         81       8841.5       5568.0  

![memcpy_in_stream](images/memcpy_in_stream.png)

<h2 style="color:#65AE11;">Check for Understanding</h2>

Please answer the following to confirm you've learned the main objectives of this section. You can display the answers for each question by clicking on the "..." cells below the questions.

---

**Knowing what you do about default stream behavior, explain why in the exercise above (and as shown in the screenshot above) we did not see any overlap between memory transfers and kernel execution, or, between host-to-device and device-to-host memory transfers, even though memory transfers were performed in non-default streams.**

**Answer:**

Commands issued into the default stream will wait until all other non-default stream commands have completed, and, will block any other non-default streams from doing work until it completes.

In the above exercise, because we did not specify a non-default stream, the kernel launch `decrypt_gpu` occured in the default stream. Therefore, it waited for the host-to-device memory transfers in the non-default stream to complete before beginning, and then, blocked the device-to-host memory transfers from beginning until it completed.

---

**Memory allocated on the host with `malloc` can be transfered asynchronously with `cudaMemcpyAsync`?**

1. True
2. False

**Answer: 2.**

Host OS memory paging, cannot guarantee the immediate location of any memory in RAM, but rather, might use paging so that the memory can be stored outside of RAM.

In order to transfer memory to or from the host asynchronously in a non-default stream, memory must be page-locked or pinned. To do this, we use `cudaMallocHost` and not `malloc`.

---

**Host code that uses data being transfered in a non-default stream will wait for memory transfers to complete before beginning work.**

1. True
2. False

**Answer: 2.**

`cudaStreamSynchronize` must be used to block host code from proceeding until work in a given stream is complete.

---

<h2 style="color:#65AE11;">Next</h2>

While you have learned how to perform kernel launches and memory transfers in non-default streams, the last 2 sections have not actually resulted in performance gains for the cipher application.

In the next sections, you will learn how to perform copy/compute overlap, and will begin to see actual performance gains from using concurrent streams.

Please continue to the next section: [*Copy Compute Considerations*](../08_Copy_Compute_Considerations/Copy_Compute_Considerations.ipynb).

<h2 style="color:#65AE11;">Optional Further Study</h2>

The following are for students with time and interest to do additional study on topics related to this workshop.

* The `async` suffix for some memcpy operations "is a misnomer as each function may exhibit synchronous or asynchronous behavior depending on the arguments passed to the function." See [the CUDA Runtime Docs](https://docs.nvidia.com/cuda/cuda-runtime-api/api-sync-behavior.html#api-sync-behavior) for more details.
* For those of you working with applications utilizing unified memory, see the answer to [this Stack Overflow answer](https://stackoverflow.com/questions/23518299/unified-memory-and-streams-in-c) (including its references to [the docs](https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#um-coherency-hd) for details.